This script processes the original <a href="http://www.countyhealthrankings.org/explore-health-rankings/rankings-data-documentation">County Health Rankings Data</a> into a format a bit more suitable for my teaching purposes.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [5]:
def friendlify(s):
    s = s.replace('^','')
    s = s.replace('\n','')
    return s

In [6]:
def read_chr_data(fn,sheet_name='Ranked Measure Data'):
    df = pd.read_excel(fn,sheet_name=sheet_name)
    top_row_values = df.columns.values
    for idx,item in enumerate(top_row_values.copy()):
        if item.startswith('Unnamed'):
            top_row_values[idx] = top_row_values[idx-1]       
    df = pd.read_excel(fn,sheet_name=sheet_name,skiprows=1)
    middle_row_values = df.columns.values
    column_names = list(middle_row_values[:3])
    top_row_values = top_row_values[3:]
    for i in range(len(top_row_values)):
        column_names.append(top_row_values[i] + ' - ' + middle_row_values[i+3].split('.')[0])        
    df = pd.read_excel(fn,sheet_name=sheet_name,skiprows=1,names=column_names,dtype={'FIPS':np.str})

    for column in df.columns.values:
        df = df.rename(columns={column:friendlify(column)})
    
    # Remove blank lines that sometimes show up on the import.
    try:
        if np.isnan(df.iloc[-1].State):
            df = df.iloc[:-1]
    except:
        _ = True
    return df

In [7]:
chr_datasets = {2010:'original_data/2010 County Health Rankings National Data_v2.xls',
                2011:'original_data/2011 County Health Rankings National Data_v2.xls',
                2012:'original_data/2012 County Health Rankings National Data_v2.xls',
                2013:'original_data/2013CountyHealthRankingsNationalData.xls',
                2014:'original_data/2014 County Health Rankings Data - v6.xls',
                2015:'original_data/2015 County Health Rankings Data - v3.xls',
                2016:'original_data/2016 County Health Rankings Data - v3.xls',
                2017:'original_data/2017CountyHealthRankingsData.xls',
                2018:'original_data/2018 County Health Rankings Data - v2.xls',
                2019:'original_data/2019 County Health Rankings Data - v1.xls'}

In [8]:
years = np.sort(list(chr_datasets.keys()))
years = [2016]

for year in years:
    print(year)
    fn = chr_datasets[year]
    df0 = read_chr_data(fn,sheet_name='Ranked Measure Data')
    
    if year > 2010:
        df1 = read_chr_data(fn,sheet_name='Additional Measure Data')

        # Drop duplicate columns in second spreadsheet and merge the tables
        df1 = df1.drop(columns=['State','County'])
        df = df0.merge(df1,how='left',on='FIPS')  
    else:
        df = df0
        
    # Make any other necessary repairs:
    # Shannon County was renamed to Oglala Lakota County in 2015
    if year >= 2015:
        df.loc[df['FIPS']=='46113','FIPS'] = '46102'
    
    # Remove columns that are confidence intervals or quartiles
    which_cols = ~(df.columns.str.contains('CI')) & ~(df.columns.str.contains('Quartile'))
    df = df.loc[:,which_cols] 
    
    # Write out the data
    df.to_csv('chr_' + str(year) + '.csv',index=False)

2016
